# Bộ định giá sản phẩm (Product Pricer) – Phần tiếp theo

Một mô hình có thể ước tính giá của một sản phẩm từ mô tả của nó.

## Quản lý dữ liệu (Data Curation) – Phần 2

Hôm nay, chúng ta sẽ mở rộng tập dữ liệu để bao phủ nhiều sản phẩm hơn, đồng thời tinh chỉnh nó thành một bộ dữ liệu tuyệt vời phục vụ cho huấn luyện mô hình.  
Việc quản lý dữ liệu có thể không hào hứng như các công việc khác mà chúng ta vẫn làm, nhưng nó là một phần quan trọng trong trách nhiệm của kỹ sư LLM và là một kỹ năng cần rèn luyện, để bạn có thể xây dựng các giải pháp thương mại với bộ dữ liệu chất lượng cao.

Bộ dữ liệu nằm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

Và thư mục chứa tất cả các bộ dữ liệu sản phẩm ở đây:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories

## Lưu ý quan trọng – vui lòng đọc trước

Chúng ta sắp tạo một bộ dữ liệu lớn gồm 400.000 mục, bao phủ nhiều loại sản phẩm khác nhau. Ở Tuần 2, chúng ta sẽ sử dụng dữ liệu này để huấn luyện mô hình riêng. Đây là một bộ dữ liệu khá lớn, và tùy thuộc vào GPU bạn chọn, quá trình huấn luyện có thể mất hơn 20 giờ. Quá trình này sẽ rất thú vị, nhưng cũng có thể tốn vài đô la tiền tài nguyên tính toán.

Ngoài ra, nếu bạn muốn tiết kiệm thời gian và chi phí, bạn có thể làm việc với một bộ dữ liệu nhỏ hơn chỉ tập trung vào thiết bị gia dụng (Home Appliances). Bạn vẫn sẽ học được đầy đủ các kiến thức cần thiết; kết quả sẽ tốt – dù không bằng bộ dữ liệu đầy đủ, nhưng vẫn rất ấn tượng! Nếu muốn, tôi đã chuẩn bị một notebook Jupyter thay thế trong thư mục này với tên `lite.ipynb` mà bạn nên sử dụng thay cho notebook này.

Bên cạnh đó, nếu bạn muốn rút ngắn quá trình xử lý dữ liệu, bạn có thể tải về các file pickle mà chúng ta lưu ở ô lệnh cuối cùng. Các file pickle có sẵn ở đây: https://drive.google.com/drive/folders/1nABNne5dl0u-TDoc_WF-jgVn88TkhUeS

In [ ]:
# imports

import os
import random
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
import numpy as np
import pickle

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

In [ ]:
# More imports after HF login

from loaders import ItemLoader
from items import Item

In [ ]:
%matplotlib inline

## Mã nguồn ItemLoader

Hãy xem trong file loaders.py – ở đó có một số đoạn mã hữu ích giúp công việc của chúng ta trở nên dễ dàng hơn.

In [ ]:
# Load in the same dataset as last time

items = ItemLoader("Appliances").load()

In [ ]:
# Look for a familiar item..
print(items[1].prompt)

## Bây giờ là mở rộng quy mô

Hãy cùng xem tất cả các bộ datasets về các items mà bạn có thể tìm thấy trong một cửa hàng bán lẻ gia dụng lớn – bao gồm electrical, electronic, office và các category liên quan, ngoại trừ clothes/ beauty/ books.

In [ ]:
dataset_names = [
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
]

In [ ]:
items = []
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load())

# Now, time for a coffee break!!
# By the way, I put the biggest datasets first.. it gets faster.

In [ ]:
print(f"A grand total of {len(items):,} items")

In [ ]:
# Plot the distribution of token counts again

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="skyblue", bins=range(0, 300, 10))
plt.show()

In [ ]:
# Plot the distribution of prices

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="blueviolet", bins=range(0, 1000, 10))
plt.show()

In [ ]:
category_counts = Counter()
for item in items:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Bar chart by category
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="goldenrod")
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

# Mục tiêu

Tạo một bộ dữ liệu cân bằng hơn về mặt giá cả. Tránh bị nghiêng quá nhiều về các sản phẩm giá rẻ, với mức giá trung bình cao hơn $60. Đồng thời cố gắng cân bằng giữa các categories – giảm số lượng items thuộc category Automotive.

In [ ]:
# Create a dict with a key of each price from $1 to $999
# And in the value, put a list of items with that price (to nearest round number)

slots = defaultdict(list)
for item in items:
    slots[round(item.price)].append(item)

In [ ]:
# Create a dataset called "sample" which tries to more evenly take from the range of prices
# And gives more weight to items from categories other than Automotive
# Set random seed for reproducibility

np.random.seed(42)
random.seed(42)
sample = []
for i in range(1, 1000):
    slot = slots[i]
    if i>=240:
        sample.extend(slot)
    elif len(slot) <= 1200:
        sample.extend(slot)
    else:
        weights = np.array([1 if item.category=='Automotive' else 5 for item in slot])
        weights = weights / np.sum(weights)
        selected_indices = np.random.choice(len(slot), size=1200, replace=False, p=weights)
        selected = [slot[i] for i in selected_indices]
        sample.extend(selected)

print(f"There are {len(sample):,} items in the sample")

In [ ]:
# Plot the distribution of prices in sample

prices = [float(item.price) for item in sample]
plt.figure(figsize=(15, 10))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# OK, we did well in terms of raising the average price and having a smooth-ish population of prices
# Let's see the categories

category_counts = Counter()
for item in sample:
    category_counts[item.category]+=1

categories = category_counts.keys()
counts = [category_counts[category] for category in categories]

# Create bar chart
plt.figure(figsize=(15, 6))
plt.bar(categories, counts, color="lightgreen")

# Customize the chart
plt.title('How many in each category')
plt.xlabel('Categories')
plt.ylabel('Count')

plt.xticks(rotation=30, ha='right')

# Add value labels on top of each bar
for i, v in enumerate(counts):
    plt.text(i, v, f"{v:,}", ha='center', va='bottom')

# Display the chart
plt.show()

In [ ]:
# Automotive still in the lead, but improved somewhat
# For another perspective, let's look at a pie

plt.figure(figsize=(12, 10))
plt.pie(counts, labels=categories, autopct='%1.0f%%', startangle=90)

# Add a circle at the center to create a donut chart (optional)
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('Categories')

# Equal aspect ratio ensures that pie is drawn as a circle
plt.axis('equal')  

plt.show()

# Bộ dữ liệu đã được quản lý!

Chúng ta đã tạo ra một bộ dữ liệu tuyệt vời.

Hãy thực hiện một số kiểm tra cuối cùng.

In [ ]:
# How does the price vary with the character count of the prompt?

sizes = [len(item.prompt) for item in sample]
prices = [item.price for item in sample]

# Create the scatter plot
plt.figure(figsize=(15, 8))
plt.scatter(sizes, prices, s=0.2, color="red")

# Add labels and title
plt.xlabel('Size')
plt.ylabel('Price')
plt.title('Is there a simple correlation?')

# Display the plot
plt.show()

In [ ]:
def report(item):
    prompt = item.prompt
    tokens = Item.tokenizer.encode(item.prompt)
    print(prompt)
    print(tokens[-10:])
    print(Item.tokenizer.batch_decode(tokens[-10:]))

In [ ]:
report(sample[398000])

## Nhận xét

Một điều thú vị về bộ tokenizer của Llama là mọi số từ 1 đến 999 đều được ánh xạ thành 1 token, tương tự như chúng ta đã thấy với gpt-4o. Điều này không đúng với qwen2, gemma và phi3, vì các model này ánh xạ từng chữ số riêng lẻ thành các token. Điều này thực sự khá hữu ích cho dự án của chúng ta, mặc dù nó không phải là một yêu cầu bắt buộc.

# Cuối cùng

Đã đến lúc chia nhỏ dữ liệu của chúng ta thành các bộ dữ liệu huấn luyện, kiểm tra và xác thực.

Thông thường, người ta sử dụng 5%-10% dữ liệu cho mục đích kiểm tra, nhưng thực tế là hiện tại chúng ta có nhiều dữ liệu hơn mức cần thiết. Chúng ta sẽ lấy 400.000 mẫu cho bộ dữ liệu huấn luyện, và dành 2.000 mẫu cho bộ kiểm tra, mặc dù thực tế sẽ không sử dụng hết số này.


In [ ]:
random.seed(42)
random.shuffle(sample)
train = sample[:400_000]
test = sample[400_000:402_000]
print(f"Divided into a training set of {len(train):,} items and test set of {len(test):,} items")

In [ ]:
print(train[0].prompt)

In [ ]:
print(test[0].test_prompt())

In [ ]:
# Plot the distribution of prices in the first 250 test points

prices = [float(item.price) for item in test[:250]]
plt.figure(figsize=(15, 6))
plt.title(f"Avg {sum(prices)/len(prices):.2f} and highest {max(prices):,.2f}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="darkblue", bins=range(0, 1000, 10))
plt.show()

# Cuối cùng – tải lên bộ dữ liệu mới của bạn

Chuyển đổi thành các prompt và tải lên HuggingFace hub.

In [ ]:
train_prompts = [item.prompt for item in train]
train_prices = [item.price for item in train]
test_prompts = [item.test_prompt() for item in test]
test_prices = [item.price for item in test]

In [ ]:
# Create a Dataset from the lists

train_dataset = Dataset.from_dict({"text": train_prompts, "price": train_prices})
test_dataset = Dataset.from_dict({"text": test_prompts, "price": test_prices})
dataset = DatasetDict({
    "train": train_dataset,
    "test": test_dataset
})

In [ ]:
# Uncomment these lines if you're ready to push to the hub, and replace my name with your HF username

# HF_USER = "kenzytran"
# DATASET_NAME = f"{HF_USER}/pricer-data"
# dataset.push_to_hub(DATASET_NAME, private=True)

In [ ]:
# One more thing!
# Let's pickle the training and test dataset so we don't have to execute all this code next time!

with open('train.pkl', 'wb') as file:
    pickle.dump(train, file)

with open('test.pkl', 'wb') as file:
    pickle.dump(test, file)

## Các việc cần làm cho bạn:

- Nghiên cứu kỹ hơn về bộ dữ liệu!
- Xác nhận rằng tokenizer phân tách tất cả các giá trị có 3 chữ số thành 1 token